In [1]:
from __future__ import print_function
import os
from skimage import io, transform
import numpy as np
from tqdm import tqdm
from model import FireNet

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

torch.backends.cudnn.benchmark = True
torch.backends.cudnn.fastest = True

In [2]:
print(torch.cuda.is_available())
net = FireNet()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.cuda()

True


FireNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (drop1): Dropout2d(p=0.5, inplace=False)
  (drop2): Dropout(p=0.2, inplace=False)
  (dense1): Linear(in_features=2304, out_features=256, bias=True)
  (dense2): Linear(in_features=256, out_features=128, bias=True)
  (dense3): Linear(in_features=128, out_features=2, bias=True)
)

In [3]:
TRAINING_PATH='Dataset'
CATEGORIES = ['Fire', 'NoFire']

In [4]:
class Rescale(object):
    def __init__(self, output_size):
        assert isinstance(output_size, tuple)
        self.output_size = output_size
        
    def __call__(self, sample):
        image, class_num = sample['image'], sample['class']
        h, w = image.shape[:2]
        new_h, new_w = self.output_size
        new_h, new_w = int(new_h), int(new_w)
        img = transform.resize(image, (new_h, new_w))
        return {'image': img, 'class': class_num}
    
class ToTensor(object):
    def __call__(self, sample):
        image, class_num = sample['image'], sample['class']
        image = image.transpose((2, 0, 1))
        return {'image': torch.from_numpy(image), 'class': class_num}

In [5]:
class TrainingSet(Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        classes = []
        images = []
        for category in CATEGORIES:
            path = os.path.join(TRAINING_PATH, category)
            class_num = CATEGORIES.index(category)
            
            for img in tqdm(os.listdir(path)):
                try:
                    image = io.imread(os.path.join(path, img))
                    if (image.shape[2] == 3):
                        images.append(image)
                        classes.append(class_num)
                except Exception as e:
                    pass
        self.set = {'image': images, 'class': classes}
    def __len__(self):
        return len(self.set['class'])
    def __getitem__(self, idx):
        image = self.set['image'][idx]
        classe = self.set['class'][idx]
        sample = {'image': image, 'class': classe}
        if self.transform:
            composed = transforms.Compose([Rescale((64, 64)), ToTensor()])
            sample = composed(sample)
        return sample
            
        

In [6]:
training_set= TrainingSet(transform=True)

100%|██| 1280/1280 [00:33<00:00, 38.20it/s]


In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

In [8]:
trainloader = DataLoader(training_set, batch_size=32, shuffle=True, num_workers=0, pin_memory=True)
net = net.float()

In [9]:
len(trainloader)

75

In [10]:
for epoch in range(100):
    global_loss = 0.0
    for i, data in tqdm(enumerate(trainloader, 0)):
        inputs, labels = data['image'].cuda(), data['class'].cuda()

        optimizer.zero_grad()

        outputs = net(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        global_loss += loss.item()
    print('global loss for epoch %d : %.3f' % (epoch + 1, global_loss/ len(trainloader)))

75it [10:45,  8.61s/it]


global loss for epoch 1 : 0.658


75it [10:45,  8.60s/it]


global loss for epoch 2 : 0.613


75it [10:57,  8.76s/it]


...

global loss for epoch 98 : 0.192


75it [10:20,  8.27s/it]


global loss for epoch 99 : 0.196


75it [10:20,  8.27s/it]


global loss for epoch 100 : 0.170


In [11]:
torch.save(net.state_dict(), './trained_weights100.pth')

In [12]:
net = FireNet()
net.float()
net.to(device)
net.load_state_dict(torch.load('./trained_weights100.pth'))

<All keys matched successfully>

In [13]:
trainloader = DataLoader(training_set, batch_size=32, shuffle=False, num_workers=0)
net = net.float()

In [14]:
correct = 0
total = 0
with torch.no_grad():
    print('evaluate accuracy on training set:')
    for data in tqdm(trainloader):
        images, labels = data['image'].to(device), data['class'].to(device)
        outputs = net(images.float())
        _, predicted = torch.max(outputs.data, 1)
        total += len(labels)
        correct += (predicted == labels).sum().item()
print ('accuracy: %d' % (100*correct/total))

evaluate accuracy on training set:


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [10:20<00:00,  8.27s/it]


accuracy: 93
